In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cap=cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame=cap.read()
    if not ret:
        break
    #img=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    k=cv2.imshow('Frame',frame)
    if cv2.waitKey(10)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [3]:
import mediapipe as mp
mp_drawing=mp.solutions.drawing_utils
mp_pose=mp.solutions.pose
mp_holistic = mp.solutions.holistic

In [5]:
mp_drawing_styles = mp.solutions.drawing_styles
cap=cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame=cap.read()
        if not ret:
            break
        
        #k=cv2.imshow('Frame',frame)
        
        image=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable=False
        result=holistic.process(image)
        image.flags.writeable=True
        image=cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if result.pose_landmarks:
            mp_drawing.draw_landmarks(
                image,
                result.pose_landmarks,
                mp_holistic.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),
            )

        # Draw Left Hand landmarks
        if result.left_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                result.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
            )

        # Draw Right Hand landmarks
        if result.right_hand_landmarks:
            mp_drawing.draw_landmarks(
                image,
                result.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_hand_landmarks_style(),
            )

        
        cv2.imshow("pose",image)
    
        
        if cv2.waitKey(10)&0xFF==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    

In [10]:
import os
poses=['punch','neutral']
frames=30
sequence=20
for i,name in enumerate(poses):
    name_path=os.makedirs(name,exist_ok=True)
    for k in range(sequence):
        video=os.makedirs(os.path.join(name,str(k)),exist_ok=True)
        

In [11]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def extract_keypoints(results):
    """
    Extract and flatten pose, left hand, and right hand landmarks from Mediapipe Holistic 'results'.
    Returns a single (n,) NumPy array.
    We'll store x, y, z for hands, and x, y, z, visibility for pose.
    Adjust as needed for your use case.
    """
    # Pose: 33 landmarks -> [x,y,z,visibility]
    if results.pose_landmarks:
        pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] 
                         for lm in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33*4)
    
    # Left hand: 21 landmarks -> [x,y,z]
    if results.left_hand_landmarks:
        lh = np.array([[lm.x, lm.y, lm.z] 
                       for lm in results.left_hand_landmarks.landmark]).flatten()
    else:
        lh = np.zeros(21*3)
    
    # Right hand: 21 landmarks -> [x,y,z]
    if results.right_hand_landmarks:
        rh = np.array([[lm.x, lm.y, lm.z] 
                       for lm in results.right_hand_landmarks.landmark]).flatten()
    else:
        rh = np.zeros(21*3)

    return np.concatenate([pose, lh, rh])

poses = ['punch', 'neutral']
sequence = 20          # Number of videos per class
frames_per_video = 30  # Number of frames per video
frame_delay = 33       # Delay between frames (ms)

# Create folders for poses and videos
for pose in poses:
    os.makedirs(pose, exist_ok=True)
    for vid in range(sequence):
        os.makedirs(os.path.join(pose, str(vid)), exist_ok=True)

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as holistic:
    
    for pose in poses:
        for vid in range(sequence):
            
            # 3-second countdown before recording each video
            for countdown in range(3, 0, -1):
                ret, frame = cap.read()
                if not ret:
                    break
                # Display which pose/video we're about to record
                cv2.putText(frame, f"Pose: {pose} (Video {vid})", 
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (255, 255, 0), 2)
                # Show countdown
                cv2.putText(frame, f"Starting in {countdown}", 
                            (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (0, 0, 255), 2)
                cv2.imshow("Pose Detection", frame)
                cv2.waitKey(1000)  # Wait 1 second per countdown tick

            frames_data = []
            for frame_num in range(frames_per_video):
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Convert BGR to RGB
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                image.flags.writeable = False
                results = holistic.process(image)
                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                # (Optional) Draw landmarks on the frame to visualize
                if results.pose_landmarks:
                    mp_drawing.draw_landmarks(
                        image, 
                        results.pose_landmarks, 
                        mp_holistic.POSE_CONNECTIONS
                    )
                if results.left_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, 
                        results.left_hand_landmarks, 
                        mp_holistic.HAND_CONNECTIONS
                    )
                if results.right_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, 
                        results.right_hand_landmarks, 
                        mp_holistic.HAND_CONNECTIONS
                    )

                # Extract the pose + hand keypoints
                keypoints = extract_keypoints(results)
                frames_data.append(keypoints)

                # Display pose/video info on every frame
                cv2.putText(image, f"Recording: Pose={pose} Video={vid}", 
                            (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 
                            1, (255, 255, 0), 2)
                
                cv2.imshow("Pose Detection", image)

                if cv2.waitKey(frame_delay) & 0xFF == ord('q'):
                    break
            
            # Convert list of frames to NumPy array, save to file
            frames_data = np.array(frames_data)
            out_path = os.path.join(pose, str(vid), f"{vid}.npy")
            np.save(out_path, frames_data)

cap.release()
cv2.destroyAllWindows()


In [12]:
import tensorflow as tf

# List all physical devices recognized as GPUs
gpus = tf.config.list_physical_devices('GPU')
print("GPUs Available: ", gpus)

# Alternatively, check if TensorFlow is built with CUDA support:
print("Built with CUDA:", tf.test.is_built_with_cuda())

# Check if GPU is being used by TensorFlow
print("Is GPU available:", tf.test.is_gpu_available())


GPUs Available:  []
Built with CUDA: False
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is GPU available: False


In [4]:
import cv2
import mediapipe as mp
import numpy as np
import os
import time
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model


poses=['punch','neutral']
frames=30
sequence=20
# ------------------ DATA COLLECTION DONE ------------------

################################################################################
# 5) LOADING THE COLLECTED DATA AND TRAINING AN LSTM MODEL ON KEYPOINTS
################################################################################

X = []
y = []
label_map = {p: idx for idx, p in enumerate(poses)} 
# e.g. {"punch": 0, "neutral": 1}

for pose in poses:
    for vid in range(sequence):
        npy_path = os.path.join(pose, str(vid), f"{vid}.npy")
        if os.path.exists(npy_path):
            data = np.load(npy_path)  # shape = (30, 258)
            # Confirm it matches (30, 258)
            if data.shape == (frames, 258):
                X.append(data)
                y.append(label_map[pose])

X = np.array(X)  # shape = (num_samples, 30, 258)
y = np.array(y)  # shape = (num_samples,)

print(f"Collected data shape: {X.shape}, labels shape: {y.shape}")
# Example: X.shape = (40, 30, 258) if 2 classes x 20 videos each

# 6) DEFINE A SIMPLE LSTM MODEL
input_keypoints = Input(shape=(frames, 258))  
# (30, 258) for each sample

x = LSTM(64, return_sequences=True)(input_keypoints)
x = LSTM(128, return_sequences=False)(x)  # only final hidden state
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_keypoints, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# 7) TRAIN THE MODEL
model.fit(X, y, epochs=10, batch_size=8)

# Done!


Collected data shape: (36, 30, 258), labels shape: (36,)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 30, 258)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 30, 64)              │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 128)                 │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 181,633 (709.50 KB)

 Trainable params: 181,633 (709.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.5017 - loss: 0.7557
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3970 - loss: 0.7581
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5179 - loss: 0.6911
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.5584 - loss: 0.6771
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5619 - loss: 0.6807
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5278 - loss: 0.6821
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.7373 - loss: 0.6514
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.6447 - loss: 0.6282
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.7072 - loss: 0.5559
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7031 - loss: 0.5241


In [5]:
poses = ['punch', 'neutral']

# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to extract pose + hand keypoints
def extract_keypoints(results):
    """
    Extract and flatten pose, left hand, and right hand landmarks from Mediapipe Holistic 'results'.
    Returns a single (258,) NumPy array.
    """
    if results.pose_landmarks:
        pose = np.array([[lm.x, lm.y, lm.z, lm.visibility] for lm in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33 * 4)
    
    if results.left_hand_landmarks:
        lh = np.array([[lm.x, lm.y, lm.z] for lm in results.left_hand_landmarks.landmark]).flatten()
    else:
        lh = np.zeros(21 * 3)
    
    if results.right_hand_landmarks:
        rh = np.array([[lm.x, lm.y, lm.z] for lm in results.right_hand_landmarks.landmark]).flatten()
    else:
        rh = np.zeros(21 * 3)

    return np.concatenate([pose, lh, rh])

# Open Webcam
cap = cv2.VideoCapture(0)

# Buffer to store the last 30 frames
sequence = []
sequence_length = 30  # Must match frames per video in training

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB for Mediapipe
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = holistic.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Draw landmarks
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
        if results.left_hand_landmarks:
            mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
        if results.right_hand_landmarks:
            mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        # Extract keypoints
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)

        # Ensure sequence is 30 frames long
        if len(sequence) > sequence_length:
            sequence.pop(0)  # Remove oldest frame

        # Make prediction when we have 30 frames
        if len(sequence) == sequence_length:
            input_data = np.expand_dims(sequence, axis=0)  # Shape: (1, 30, 258)
            prediction = model.predict(input_data)[0][0]  # Output probability
            
            # Convert to class label
            if prediction > 0.5:
                label = poses[1]  # "neutral"
            else:
                label = poses[0]  # "punch"

            # Display prediction on the frame
            cv2.putText(image, f"Prediction: {label} ({prediction:.2f})", 
                        (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 
                        1, (0, 255, 0), 2)

        # Show Webcam Output
        cv2.imshow("Pose Detection", image)

        # Press 'q' to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━